In [1]:
# Datamart server running on localhost in development mode
datamart_api_url = 'http://localhost:5000'

In [13]:
from requests import get,post,put,delete
import json
import pandas as pd
from io import StringIO
from IPython.display import display, HTML

In [9]:
import os
def upload_data_post(file_path, url):
    file_name = os.path.basename(file_path)
    files = {
        'file': (file_name, open(file_path, mode='rb'), 'application/octet-stream')
    }
    response = post(url, files=files)

    print(json.dumps(response.json(), indent=2))

In [18]:
response = get(f'{datamart_api_url}/metadata/datasets/DeleteMe/variables')
df = pd.DataFrame(response.json())
df

,name,variable_id,description,corresponds_to_property,qualifier
0,INGO,ingo,INGO in DeleteMe,PVARIABLE-QDeleteMe-003,"[{'name': 'Location', 'identifier': 'PQUALIFIE..."


In [7]:
response = get(f'{datamart_api_url}/metadata/datasets/DeleteMe/variables/DeleteV01')
print(json.dumps(response.json(), indent=2))

{
  "name": "test variable for delete dataset",
  "variable_id": "DeleteV01",
  "dataset_id": "DeleteMe",
  "description": "test variable for delete dataset",
  "corresponds_to_property": "PDeleteMe-DeleteV01"
}


In [26]:
url = f'{datamart_api_url}/datasets/DeleteMe/annotated'
file_path = 'test/test_data/03_worker_incidents.xlsx'
upload_data_post(file_path, url)

[
  {
    "name": "UN",
    "variable_id": "un",
    "dataset_id": "DeleteMe",
    "description": "UN in DeleteMe",
    "corresponds_to_property": "PVARIABLE-QDeleteMe-003",
    "qualifier": [
      {
        "name": "located in the administrative territorial entity",
        "identifier": "P131"
      },
      {
        "name": "stated in",
        "identifier": "P248"
      },
      {
        "name": "point in time",
        "identifier": "P585"
      }
    ]
  },
  {
    "name": "INGO",
    "variable_id": "ingo",
    "dataset_id": "DeleteMe",
    "description": "INGO in DeleteMe",
    "corresponds_to_property": "PVARIABLE-QDeleteMe-004",
    "qualifier": [
      {
        "name": "located in the administrative territorial entity",
        "identifier": "P131"
      },
      {
        "name": "stated in",
        "identifier": "P248"
      },
      {
        "name": "point in time",
        "identifier": "P585"
      }
    ]
  }
]


In [39]:
response = get(f'{datamart_api_url}/datasets/DeleteMe/variables/un')
df = pd.read_csv(StringIO(response.text))
display(HTML(df.fillna('').to_html(index=False)))

location_id,dataset_id,variable_id,variable,main_subject,main_subject_id,value,value_unit,time,time_precision,country,admin1,admin2,admin3,coordinate,stated_in,stated_in_id,location,stated in


In [37]:
response = delete(f'{datamart_api_url}/datasets/DeleteMe/variables')
print(response.text)
# print(json.dumps(response.json(), indent=2))

{"message": "The method is not allowed for the requested URL."}



In [48]:
response = delete(f'{datamart_api_url}/metadata/datasets/DeleteMe/variables/ingo')
print(response.text)
# print(json.dumps(response.json(), indent=2))

{"Message": "Variable: ingo in the dataset: DeleteMe has been deleted."}



In [49]:
response = get(f'{datamart_api_url}/metadata/datasets/DeleteMe/variables')
print(len(response.json()))
print(json.dumps(response.json(), indent=2)) # print only 5 variables

0
[]
